# Tutorial 1: Node Classification with DGL

Source: https://docs.dgl.ai/tutorials/blitz/1_introduction.html#sphx-glr-tutorials-blitz-1-introduction-py

In [1]:
import os
os.environ['DGLBACKEND'] = 'pytorch'
import torch 
import torch.nn as nn
import torch.nn.functional as F

import dgl
import dgl.data

In [2]:
# Data loading 
data = dgl.data.CoraGraphDataset()

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [3]:
print(f"Number of categories: {data.num_classes}")

Number of categories: 7


In [4]:
data

Dataset("cora_v2", num_graphs=1, save_path=/home/dloader-ubuntu/.dgl/cora_v2)

In [5]:
g = data[0]

In [6]:
g

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})

In [7]:
print(f"Node features: {g.ndata}. Edge features: {g.edata}")

Node features: {'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False])}. Edge features: {}


In [8]:
from dgl.nn import GraphConv

In [9]:
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)
        
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g,h)
        return h
    
# Create the GCN model 
model = GCN(g.ndata['feat'].shape[1], 16, data.num_classes)


In [10]:
g

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})

In [15]:
# GCN Training
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0
    
    # Get variables from the graph
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    
    for e in range(100):
        # Forward pass
        logits = model(g, features)
        
        # Compute the prediction
        pred = logits.argmax(1)
        
        # Compute the loss
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])
        
        # Compute the accuracy on training/validation/test set
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()
        
        # Save the best validation acc and the test accuracy
        # print(f"Validation accuracy: {val_acc:.4f} Test accuracy: {test_acc:.4f}")
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc
            
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if e % 5 == 0:
            print(f"In epoch {e}, loss: {loss:.4f}, validation accuracy (best): {val_acc:.4f} ({best_val_acc:.4f}), test accuracy (best): {test_acc:.4f} ({best_test_acc:.4f})")

In [16]:
model = GCN(g.ndata['feat'].shape[1], 16, data.num_classes)
train(g, model)

In epoch 0, loss: 1.9458, validation accuracy (best): 0.1300 (0.1300), test accuracy (best): 0.1400 (0.1400)
In epoch 5, loss: 1.8856, validation accuracy (best): 0.5740 (0.5940), test accuracy (best): 0.5880 (0.6040)
In epoch 10, loss: 1.7987, validation accuracy (best): 0.6300 (0.6300), test accuracy (best): 0.6400 (0.6490)
In epoch 15, loss: 1.6873, validation accuracy (best): 0.6300 (0.6300), test accuracy (best): 0.6370 (0.6490)
In epoch 20, loss: 1.5532, validation accuracy (best): 0.6660 (0.6660), test accuracy (best): 0.6700 (0.6700)
In epoch 25, loss: 1.4000, validation accuracy (best): 0.6880 (0.6880), test accuracy (best): 0.6940 (0.6940)
In epoch 30, loss: 1.2345, validation accuracy (best): 0.7140 (0.7140), test accuracy (best): 0.7070 (0.7070)
In epoch 35, loss: 1.0653, validation accuracy (best): 0.7240 (0.7240), test accuracy (best): 0.7210 (0.7160)
In epoch 40, loss: 0.9011, validation accuracy (best): 0.7340 (0.7340), test accuracy (best): 0.7330 (0.7330)
In epoch 45,

## Training on The GPU

In [17]:
# Move the graph to the GPU
g = g.to('cuda:0')
model = GCN(g.ndata['feat'].shape[1], 16, data.num_classes).to('cuda:0')
train(g, model)

/home/dloader-ubuntu/miniconda3/envs/dgl-tutorial-env/lib/python3.11/site-packages/dgl/backend/pytorch/tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


In epoch 0, loss: 1.9453, validation accuracy (best): 0.1220 (0.1220), test accuracy (best): 0.1460 (0.1460)
In epoch 5, loss: 1.8909, validation accuracy (best): 0.5020 (0.5020), test accuracy (best): 0.5080 (0.5080)
In epoch 10, loss: 1.8091, validation accuracy (best): 0.5740 (0.5740), test accuracy (best): 0.6130 (0.6130)
In epoch 15, loss: 1.7048, validation accuracy (best): 0.5960 (0.5960), test accuracy (best): 0.6370 (0.6370)
In epoch 20, loss: 1.5788, validation accuracy (best): 0.6380 (0.6380), test accuracy (best): 0.6820 (0.6820)
In epoch 25, loss: 1.4340, validation accuracy (best): 0.6660 (0.6660), test accuracy (best): 0.7140 (0.7140)
In epoch 30, loss: 1.2746, validation accuracy (best): 0.6880 (0.6880), test accuracy (best): 0.7340 (0.7340)
In epoch 35, loss: 1.1075, validation accuracy (best): 0.6940 (0.6940), test accuracy (best): 0.7470 (0.7390)
In epoch 40, loss: 0.9414, validation accuracy (best): 0.7120 (0.7120), test accuracy (best): 0.7510 (0.7510)
In epoch 45,